# SimpleRNN을 이용한 텍스트 생성하기

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## 데이터 전처리

In [26]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print(f"단어 집합의 크기 : {vocab_size}")

단어 집합의 크기 : 12


In [28]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


샘플 나누기

In [29]:
sequences = list()
for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print("학습에 사용할 샘플의 개수: %d" % len(sequences))

학습에 사용할 샘플의 개수: 11


In [30]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [31]:
max_len = max(len(l) for l in sequences)

print(f"샘플의 최대 길이: {max_len}")

샘플의 최대 길이: 6


샘플 패딩 진행

In [32]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') 

In [33]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


각 샘플의 마지막 단어를 y의 샘플로 지정

In [34]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [35]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


y를 원 핫 인코딩

In [36]:
y = to_categorical(y, num_classes = vocab_size)

In [37]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 모델 설계하기

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

하이퍼파라미터인 임베딩 벡터의 차원은 10, 은닉 상태의 크기는 32로 설정
다 대 일 구조의 RNN을 사용

In [ ]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 200, verbose=2)

(모델, 토크나이저, 현재 단어, 반복할 횟수)를 입력으로 다음 단어를 예측하는 (문장을 생성하는) 함수

In [46]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen = 5, padding = 'pre')

        result = model.predict(encoded, verbose = 0)
        result = np.argmax(result, axis = 1)

        for word, index in tokenizer.word_index.items():

            if index == result:
                break
        current_word = current_word + ' '+ word

        sentence = sentence + ' ' + word 
    sentence = init_word + sentence
    return sentence        

In [47]:
print(sentence_generation(model, tokenizer, "경마장에", 4))

경마장에 있는 말이 뛰고 있다


In [48]:
print(sentence_generation(model, tokenizer, "그의", 2))

그의 말이 법이다


In [49]:
print(sentence_generation(model, tokenizer, "가는", 5))

가는 말이 고와야 오는 말이 곱다


# 글자 단위 RNN 언어 모델(Char RNNLM)

## 데이터 전처리

In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [1]:
raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''
     

In [3]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [4]:
# 중복을 제거한 글자 집합 생성
char_vocab = sorted(list(set(raw_text)))
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [5]:
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


In [6]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


In [7]:
length = 11
sequences = []
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


In [8]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [9]:
encoded_sequences = []
for sequence in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    encoded_sequence = [char_to_index[char] for char in sequence] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    encoded_sequences.append(encoded_sequence)

In [11]:
encoded_sequences[:5]

[[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18],
 [0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28],
 [16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17],
 [14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0],
 [28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]]

In [12]:
encoded_sequences = np.array(encoded_sequences)
X_data = encoded_sequences[:,:-1]

# 맨 마지막 위치의 글자를 분리
y_data = encoded_sequences[:,-1]

In [13]:
print(X_data[:5])
print(y_data[:5])

[[ 8  0 16 14 28  0 24 23  0 31]
 [ 0 16 14 28  0 24 23  0 31 18]
 [16 14 28  0 24 23  0 31 18 28]
 [14 28  0 24 23  0 31 18 28 17]
 [28  0 24 23  0 31 18 28 17  0]]
[18 28 17  0 21]


In [14]:
# 원-핫 인코딩
X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

In [15]:
print(X_data_one_hot.shape) #(샘플 수, 샘플의 길이, 문자 집합의 수)

(426, 10, 33)


## 모델 설계하기

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

은닉 상태의 크기는 64로 지정

In [17]:
hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs=100, verbose=2)

Epoch 1/100
14/14 - 8s - loss: 3.4744 - accuracy: 0.0845 - 8s/epoch - 564ms/step
Epoch 2/100
14/14 - 0s - loss: 3.3670 - accuracy: 0.1761 - 52ms/epoch - 4ms/step
Epoch 3/100
14/14 - 0s - loss: 3.1174 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 4/100
14/14 - 0s - loss: 3.0122 - accuracy: 0.1972 - 53ms/epoch - 4ms/step
Epoch 5/100
14/14 - 0s - loss: 2.9655 - accuracy: 0.1972 - 50ms/epoch - 4ms/step
Epoch 6/100
14/14 - 0s - loss: 2.9395 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 7/100
14/14 - 0s - loss: 2.9210 - accuracy: 0.1972 - 53ms/epoch - 4ms/step
Epoch 8/100
14/14 - 0s - loss: 2.8993 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 9/100
14/14 - 0s - loss: 2.8755 - accuracy: 0.1972 - 51ms/epoch - 4ms/step
Epoch 10/100
14/14 - 0s - loss: 2.8497 - accuracy: 0.1972 - 71ms/epoch - 5ms/step
Epoch 11/100
14/14 - 0s - loss: 2.8219 - accuracy: 0.1972 - 52ms/epoch - 4ms/step
Epoch 12/100
14/14 - 0s - loss: 2.7864 - accuracy: 0.1972 - 51ms/epoch - 4ms/step
Epoch 13/100
14/14 - 0s -

In [19]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):

    # 초기 시퀀스
    init_text = seed_text
    sentence = ''

    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        
        for char, index in char_to_index.items():
            if index == result:
                break

        # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        seed_text = seed_text + char

        # 예측 글자를 문장에 저장
        sentence = sentence + char

    sentence = init_text + sentence
    return sentence
     

In [21]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to hang out with programming and deep learning.
